# Create the Fetures Group from PCAPs

In [1]:
!pip install numpy pandas scapy



In [2]:
pip install pyshark


Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install lxml --force-reinstall



  Using cached lxml-4.9.3-cp38-cp38-macosx_10_9_x86_64.whl
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.3
    Uninstalling lxml-4.9.3:
      Successfully uninstalled lxml-4.9.3


In [9]:
conda install -c conda-forge nest_asyncio


Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - nest_asyncio

Current channels:

  - https://conda.anaconda.org/conda-forge/osx-arm64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/osx-arm64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-arm64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [10]:
import nest_asyncio
nest_asyncio.apply()


In [17]:
import pyshark
import pandas as pd
import nest_asyncio
from collections import defaultdict

# Apply the nest_asyncio patch
nest_asyncio.apply()

flows = defaultdict(lambda: {"packet_count": 0, "total_bytes": 0, "start_time": None, "end_time": None})
auth_attempts = []

def extract_packet_info(packet):
    eth_src = packet.eth.src
    eth_dst = packet.eth.dst
    src_ip = None
    dst_ip = None
    src_port = None
    dst_port = None
    protocol = None
    interval_between_auth_attempts = None
    flow_identification = None
    packet_count_in_flow = None
    total_bytes_transmitted = None
    traffic_direction = None
    flow_start_timestamp = None
    flow_end_timestamp = None
    tcp_http_headers = None
    http_status_codes = None
    ssh_protocol_details = None
    payload_size = None
    filtered_payload_content = None

    if 'IP' in packet:
        ip = packet.ip
        src_ip = ip.src
        dst_ip = ip.dst

        if "TCP" in packet or "UDP" in packet:
            protocol = "TCP" if "TCP" in packet else "UDP"
            layer = packet.tcp if "TCP" in packet else packet.udp
            src_port = layer.srcport
            dst_port = layer.dstport
            flow_key = f"{src_ip}:{src_port} -> {dst_ip}:{dst_port}"
            flow_identification = flow_key

            # Update flow information
            flow = flows[flow_key]
            flow["packet_count"] += 1
            flow["total_bytes"] += int(packet.length)
            if flow["start_time"] is None:
                flow["start_time"] = packet.sniff_time
            flow["end_time"] = packet.sniff_time

            # Calculate values for output
            packet_count_in_flow = flow["packet_count"]
            total_bytes_transmitted = flow["total_bytes"]
            flow_start_timestamp = flow["start_time"].strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            flow_end_timestamp = flow["end_time"].strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            traffic_direction = "Inbound" if src_ip == "your_source_ip" else "Outbound"

            if "HTTP" in packet:
                tcp_http_headers = str(packet.http)
                http_layer = packet.http
                request_method = getattr(http_layer, 'request_method', None)
                request_uri = getattr(http_layer, 'request_uri', None)
        
                # Check for authentication attempts (example: HTTP POST to login URL)
                if request_method == "POST" and request_uri and "/login" in request_uri:
                    auth_attempts.append(packet.sniff_time)
                    if len(auth_attempts) > 1:
                        interval_between_auth_attempts = (auth_attempts[-1] - auth_attempts[-2]).total_seconds()


        elif "UDP" in packet:
            protocol = "UDP"
            src_port = packet.udp.srcport
            dst_port = packet.udp.dstport

    timestamp = packet.sniff_time.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

    return {
        "eth_src": eth_src,
        "eth_dst": eth_dst,
        "src_ip": src_ip,
        "dst_ip": dst_ip,
        "src_port": src_port,
        "dst_port": dst_port,
        "protocol": protocol,
        "timestamp": timestamp,
        "interval_between_auth_attempts": interval_between_auth_attempts,
        "tcp_http_headers": tcp_http_headers,
        "flow_identification": flow_identification,
        "packet_count_in_flow": packet_count_in_flow,
        "total_bytes_transmitted": total_bytes_transmitted,
        "traffic_direction": traffic_direction,
        "flow_start_timestamp": flow_start_timestamp,
        "flow_end_timestamp": flow_end_timestamp,
        "tcp_http_headers": tcp_http_headers,
        "http_status_codes": http_status_codes,
        "ssh_protocol_details": ssh_protocol_details,
        "payload_size": payload_size,
        "filtered_payload_content": filtered_payload_content,
    }

# Function to process a list of packets
def process_packets(packets, output_csv, first_chunk=True):
    packet_info_list = [extract_packet_info(packet) for packet in packets]
    df = pd.DataFrame(packet_info_list)
    if first_chunk:
        df.to_csv(output_csv, index=False)
    else:
        df.to_csv(output_csv, index=False, mode='a', header=False)

# Function to read packets from pcap in chunks and process them
def process_pcap_in_chunks(input_pcap, output_csv, chunk_size=10000):
    cap = pyshark.FileCapture(input_pcap)
    packets = []
    first_chunk = True
    for packet in cap:
        packets.append(packet)
        if len(packets) == chunk_size:
            process_packets(packets, output_csv, first_chunk)
            packets = []
            if first_chunk:
                first_chunk = False
    if packets:  # Process remaining packets if any
        process_packets(packets, output_csv, first_chunk)

# Replace 'your_input.pcap' with your pcap file path
input_pcap = 'Tuesday-WorkingHours.pcap'
output_csv = 'output.csv'
process_pcap_in_chunks(input_pcap, output_csv)



